In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
#from sklearn import datasets
import pandas as pd

In [ ]:
## 

In [ ]:
#DS: damage state (according to the given schema)
#DMS_VS: vertical structure
#DMS_FL: floors
#DMS_RF: roof
#DMS_IP: wall partitions
#DWL_L: water penetration limited to first floor
#DWL_E: water penetration over the first floor

#check one input file
dc_ems98,lrmodel_ems98 = compute_damage_conversion("damage_conversion_test_ems98_01.csv")
dc_ems98.head()

In [ ]:
#dc_supp,lrmodel_supp = compute_damage_conversion("damage_conversion_test_supparsi2013_01.csv")
#dc_supp.head()

## Main Routine

In [2]:
from random import randint
def gen_rand_od():
    a = [randint(0,5) for p in range(0, 4)]
    a = a +([randint(0,1) for p in range(0, 2)])
    return(a)
#test
#gen_rand_fm()

def compute_damage_conversion(infile):
    dc = pd.read_csv(infile)
    dc.fillna(0,inplace=True)
    X1 = dc.iloc[:,1:]
    Y1 = dc.iloc[:,0]
    logit_reg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
    logit_reg.fit(X1, Y1)
    return (dc, logit_reg)
#test
#dc_ems98m,lrmodel = compute_damage_conversion("damage_conversion_test_ems98_01.csv")

def get_damage_conv(ds,target='B'):
    tab_target = B_given_od
    tab_source = A_given_od
    if target=='A':
        tab_target = A_given_od
        tab_source = B_given_od
        if not ds in set(ds_B):
            Raise('Error, damage state not compatible')
    elif not ds in set(ds_A):
            Raise('Error, damage state not compatible')
    res1 = tab_target.transpose().mul(tab_source[ds]*p_od).transpose()
    res2 = np.sum(res1/tab_source.mean()[ds])
    return res2
#test
#get_damage_conv(d,'B')

import json
def get_model_dict(sourcename,targetname,source_ds,target_ds,conv_matrix):
    BtoA_model = {}
    BtoA_model['source_name'] = sourcename
    BtoA_model['target_name'] = targetname
    BtoA_model['source_damage_states'] = source_ds.tolist()
    BtoA_model['target_damage_states'] = target_ds.tolist()
    BtoA_model['conv_matrix'] = conv_matrix
    return BtoA_model
#test
#get_model_dict('ems_98','sup_13',ds_A,ds_B,AtoB_conv_matrix,'test_out_ems98toSupparsi2013.json')

def save_modeldict(model,outfile):
    if outfile != "":
        model['conv_matrix'] = model['conv_matrix'].to_json()
        with open(outfile, 'w') as fp:
            json.dump(model,fp)

def load_model_dict(infile):
    with open(infile) as f:
        model = json.loads(f.read())
    model['conv_matrix'] = pd.read_json(model['conv_matrix'])
    return model
#test
#load_model_dict('test_out_ems98toSupparsi2013.json')

infile_A = "damage_conversion_test_ems98_01.csv"
infile_B = "damage_conversion_test_supparsi2013_01.csv"

#read damage conversion input and fit a logit model
dc_A,lrmodel_A = compute_damage_conversion(infile_A)
dc_B,lrmodel_B = compute_damage_conversion(infile_B)
if not np.alltrue(dc_A.columns == dc_B.columns):
    Raise('Error, Damage observation do not match')
#get damage states
ds_A  = dc_A.DS.unique()
ds_B  = dc_B.DS.unique()

#generate an exhaustive set of combinations of observable damages
set_od = pd.DataFrame([gen_rand_od() for p in range(0, 100000)]).drop_duplicates()
set_od.columns = dc_A.columns[1:]
p_od = 1/len(set_od)

#compute conditional probabilities
A_given_od = pd.DataFrame(lrmodel_A.predict_proba(set_od))
B_given_od = pd.DataFrame(lrmodel_B.predict_proba(set_od))

AtoB_conv_matrix = pd.DataFrame([get_damage_conv(d,'B') for d in ds_A])
BtoA_conv_matrix = pd.DataFrame([get_damage_conv(d,'A') for d in ds_B])

model_dict_AtoB = get_model_dict('ems_98','sup_13',ds_A,ds_B,AtoB_conv_matrix)
model_dict_BtoA = get_model_dict('sup_13','ems_98',ds_B,ds_A,BtoA_conv_matrix)

save_modeldict(model_dict_AtoB,'test_out_EMS98toSupparsi2013.json')
save_modeldict(model_dict_BtoA,'test_out_Supparsi2013toEMS98.json')


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [3]:
#sample from possible observed damage and compute expected damage state probability
set_od_ss = set_od.sample(100)
test_res = pd.DataFrame(lrmodel_A.predict_proba(set_od_ss))
#set_od_ss.head()
test_res.join(set_od_ss.reset_index())

,DMS_VS,DMS_FL,DMS_RF,DMS_IP,DWP_L,DWP_E
1106,1,4,5,4,1,0
3097,0,0,1,2,1,0
489,4,2,1,4,0,0
22976,2,5,5,2,1,1
4700,2,5,5,1,0,0
